<a href="https://colab.research.google.com/github/KostasNikolaou23/Data-Mining/blob/main/Version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time
import numpy as np

# Έλεγχος GPU [cite: 58]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Γενική συνάρτηση που τρέχει ένα πείραμα (Training + Testing)
def run_experiment(model, train_loader, test_loader, num_epochs, learning_rate, model_name):
    criterion = nn.CrossEntropyLoss()
    # Χρησιμοποιούμε Adam όπως ζητείται [cite: 56]
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"--- Training {model_name} ---")
    model = model.to(device) # Μεταφορά μοντέλου στην GPU

    # --- Training Loop ---
    start_train = time.time() # Έναρξη χρονομέτρησης εκπαίδευσης

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            # Μετατροπή διαστάσεων για MLP αν χρειάζεται (flatten)
            if model_name == 'MLP':
                images = images.reshape(-1, 28*28).to(device)
            else:
                images = images.to(device)

            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    end_train = time.time() # Λήξη χρονομέτρησης εκπαίδευσης
    train_time = end_train - start_train

    # --- Testing Loop ---
    model.eval()
    correct = 0
    total = 0

    start_test = time.time() # Έναρξη χρονομέτρησης test

    with torch.no_grad():
        for images, labels in test_loader:
            if model_name == 'MLP':
                images = images.reshape(-1, 28*28).to(device)
            else:
                images = images.to(device)

            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    end_test = time.time() # Λήξη χρονομέτρησης test
    test_time = end_test - start_test

    accuracy = 100 * correct / total
    print(f"Results for {model_name}: Acc={accuracy:.2f}%, Train Time={train_time:.2f}s, Test Time={test_time:.2f}s")

    return accuracy, train_time, test_time

Using device: cuda


In [9]:
batch_size = 100 #

# Transform για MLP και CNN
transform_simple = transforms.Compose([transforms.ToTensor()])

# Transform για MobileNetV2 (από το αρχείο mnist_MobileNetV2.py)
transform_mobile = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Loaders για MLP/CNN
train_loader_simple = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform_simple), batch_size=batch_size, shuffle=True)
test_loader_simple = DataLoader(datasets.MNIST('./data', train=False, download=True, transform=transform_simple), batch_size=batch_size, shuffle=False)

# Loaders για MobileNetV2
train_loader_mobile = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform_mobile), batch_size=batch_size, shuffle=True)
test_loader_mobile = DataLoader(datasets.MNIST('./data', train=False, download=True, transform=transform_mobile), batch_size=batch_size, shuffle=False)

In [10]:
# --- MLP Model (from mnist_MLP.py) ---
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out # H CrossEntropyLoss στο PyTorch περιλαμβάνει το LogSoftmax, οπότε επιστρέφουμε raw logits για σταθερότητα

# --- CNN Model (from mnist_CNN.py) ---
class CNNet(nn.Module):
    def __init__(self):
        super(CNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1) # [cite: 31]
        self.conv2 = nn.Conv2d(32, 64, 3, 1) # [cite: 38]
        # Υπολογισμός output μετά από convolutions και pools:
        # 28x28 -> conv1 -> 26x26 -> pool -> 13x13
        # 13x13 -> conv2 -> 11x11 -> pool -> 5x5
        # 64 κανάλια * 5 * 5 = 1600
        self.fc1 = nn.Linear(1600, 10) # [cite: 45]

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

# --- MobileNetV2 Helper ---
def get_mobilenet():
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
    # Πάγωμα παραμέτρων [cite: 72]
    for param in model.features.parameters():
        param.requires_grad = False
    # Αλλαγή ταξινομητή
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
    return model

In [11]:
results = {'MLP': [], 'CNN': [], 'MobileNet': []}

# --- 1. MLP Loop ---
print("\n=== Running MLP Experiments ===")
for i in range(5):
    print(f"Run {i+1}/5")
    model = NeuralNet(input_size=784, hidden_size=100, num_classes=10) # [cite: 27]
    acc, tr_time, te_time = run_experiment(model, train_loader_simple, test_loader_simple, num_epochs=5, learning_rate=0.001, model_name='MLP')
    results['MLP'].append((acc, tr_time, te_time))

# --- 2. CNN Loop ---
print("\n=== Running CNN Experiments ===")
for i in range(5):
    print(f"Run {i+1}/5")
    model = CNNet()
    acc, tr_time, te_time = run_experiment(model, train_loader_simple, test_loader_simple, num_epochs=5, learning_rate=0.001, model_name='CNN')
    results['CNN'].append((acc, tr_time, te_time))

# --- 3. MobileNetV2 Loop ---
print("\n=== Running MobileNetV2 Experiments ===")
for i in range(5):
    print(f"Run {i+1}/5")
    model = get_mobilenet()
    acc, tr_time, te_time = run_experiment(model, train_loader_mobile, test_loader_mobile, num_epochs=3, learning_rate=0.001, model_name='MobileNetV2')
    results['MobileNet'].append((acc, tr_time, te_time))


=== Running MLP Experiments ===
Run 1/5
--- Training MLP ---
Results for MLP: Acc=96.82%, Train Time=36.11s, Test Time=1.00s
Run 2/5
--- Training MLP ---
Results for MLP: Acc=97.04%, Train Time=36.72s, Test Time=1.40s
Run 3/5
--- Training MLP ---
Results for MLP: Acc=97.14%, Train Time=36.71s, Test Time=1.03s
Run 4/5
--- Training MLP ---
Results for MLP: Acc=97.01%, Train Time=35.64s, Test Time=0.99s
Run 5/5
--- Training MLP ---
Results for MLP: Acc=97.20%, Train Time=35.07s, Test Time=0.99s

=== Running CNN Experiments ===
Run 1/5
--- Training CNN ---
Results for CNN: Acc=98.65%, Train Time=38.67s, Test Time=1.08s
Run 2/5
--- Training CNN ---
Results for CNN: Acc=98.73%, Train Time=38.02s, Test Time=1.09s
Run 3/5
--- Training CNN ---
Results for CNN: Acc=98.97%, Train Time=37.95s, Test Time=1.46s
Run 4/5
--- Training CNN ---
Results for CNN: Acc=98.68%, Train Time=38.21s, Test Time=1.09s
Run 5/5
--- Training CNN ---
Results for CNN: Acc=98.62%, Train Time=38.61s, Test Time=1.10s

===

100%|██████████| 13.6M/13.6M [00:00<00:00, 107MB/s] 


--- Training MobileNetV2 ---
Results for MobileNetV2: Acc=95.84%, Train Time=342.63s, Test Time=26.37s
Run 2/5
--- Training MobileNetV2 ---
Results for MobileNetV2: Acc=96.13%, Train Time=350.92s, Test Time=27.18s
Run 3/5
--- Training MobileNetV2 ---
Results for MobileNetV2: Acc=95.79%, Train Time=347.88s, Test Time=26.20s
Run 4/5
--- Training MobileNetV2 ---
Results for MobileNetV2: Acc=95.92%, Train Time=338.77s, Test Time=26.19s
Run 5/5
--- Training MobileNetV2 ---
Results for MobileNetV2: Acc=95.89%, Train Time=336.83s, Test Time=26.24s


In [12]:
print("\n\n=== Final Results (Mean ± Std Dev) over 5 runs ===")
for name, data in results.items():
    data = np.array(data) # shape (5, 3) -> col 0: acc, col 1: train_time, col 2: test_time

    mean_acc = np.mean(data[:, 0])
    std_acc = np.std(data[:, 0])
    mean_train_time = np.mean(data[:, 1])
    mean_test_time = np.mean(data[:, 2])

    print(f"Model: {name}")
    print(f"  Accuracy: {mean_acc:.2f}% ± {std_acc:.2f}")
    print(f"  Avg Training Time: {mean_train_time:.2f} sec")
    print(f"  Avg Test Time: {mean_test_time:.2f} sec")
    print("-" * 30)



=== Final Results (Mean ± Std Dev) over 5 runs ===
Model: MLP
  Accuracy: 97.04% ± 0.13
  Avg Training Time: 36.05 sec
  Avg Test Time: 1.08 sec
------------------------------
Model: CNN
  Accuracy: 98.73% ± 0.13
  Avg Training Time: 38.29 sec
  Avg Test Time: 1.16 sec
------------------------------
Model: MobileNet
  Accuracy: 95.91% ± 0.12
  Avg Training Time: 343.41 sec
  Avg Test Time: 26.43 sec
------------------------------


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time
import numpy as np

# Re-defining run_experiment with the fix, as per strict instruction to modify ONLY this cell.
# The original 'device' variable from the first cell will be captured from the global scope.

# Γενική συνάρτηση που τρέχει ένα πείραμα (Training + Testing)
def run_experiment(model, train_loader, test_loader, num_epochs, learning_rate, model_name):
    global device # Declare 'device' as global to ensure the function uses the globally set device
    criterion = nn.CrossEntropyLoss()
    # Χρησιμοποιούμε Adam όπως ζητείται [cite: 56]
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"--- Training {model_name} ---")
    model = model.to(device) # Μεταφορά μοντέλου στην GPU

    # --- Training Loop ---
    start_train = time.time() # Έναρξη χρονομέτρησης εκπαίδευσης

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            # Μετατροπή διαστάσεων για MLP αν χρειάζεται (flatten)
            # FIXED: Changed 'model_name == 'MLP'' to 'model_name.startswith('MLP')'
            if model_name.startswith('MLP'):
                images = images.reshape(-1, 28*28).to(device)
            else:
                images = images.to(device)

            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    end_train = time.time() # Λήξη χρονομέτρησης εκπαίδευσης
    train_time = end_train - start_train

    # --- Testing Loop ---
    model.eval()
    correct = 0
    total = 0

    start_test = time.time() # Έναρξη χρονομέτρησης test

    with torch.no_grad():
        for images, labels in test_loader:
            # FIXED: Changed 'model_name == 'MLP'' to 'model_name.startswith('MLP')'
            if model_name.startswith('MLP'):
                images = images.reshape(-1, 28*28).to(device)
            else:
                images = images.to(device)

            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    end_test = time.time() # Λήξη χρονομέτρησης test
    test_time = end_test - start_test

    accuracy = 100 * correct / total
    print(f"Results for {model_name}: Acc={accuracy:.2f}%, Train Time={train_time:.2f}s, Test Time={test_time:.2f}s")

    return accuracy, train_time, test_time

print("\n=== Ερώτημα 2: CPU Execution Benchmark (Full Run) ===")

# Αλλαγή σε CPU
device = torch.device("cpu")
print(f"Temporarily changed device to: {device}")

# 1. MLP on CPU (5 Epochs)
print("--> Running MLP on CPU...")
model_mlp_cpu = NeuralNet(784, 100, 10)
run_experiment(model_mlp_cpu, train_loader_simple, test_loader_simple, num_epochs=5, learning_rate=0.001, model_name='MLP_CPU')

# 2. CNN on CPU (5 Epochs)
print("--> Running CNN on CPU...")
model_cnn_cpu = CNNet()
run_experiment(model_cnn_cpu, train_loader_simple, test_loader_simple, num_epochs=5, learning_rate=0.001, model_name='CNN_CPU')

# 3. MobileNetV2 on CPU (3 Epochs)
print("--> Running MobileNetV2 on CPU (This may take 1-2 hours)...")
model_mobile_cpu = get_mobilenet()
run_experiment(model_mobile_cpu, train_loader_mobile, test_loader_mobile, num_epochs=3, learning_rate=0.001, model_name='MobileNet_CPU')

# Επαναφορά σε GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Switched back to device: {device}")



=== Ερώτημα 2: CPU Execution Benchmark (Full Run) ===
Temporarily changed device to: cpu
--> Running MLP on CPU...
--- Training MLP_CPU ---
Results for MLP_CPU: Acc=97.00%, Train Time=35.47s, Test Time=0.95s
--> Running CNN on CPU...
--- Training CNN_CPU ---
Results for CNN_CPU: Acc=98.81%, Train Time=204.02s, Test Time=3.38s
--> Running MobileNetV2 on CPU (This may take 1-2 hours)...
--- Training MobileNet_CPU ---
